In [2]:
import pandas as pd
gbig_test_df = pd.read_csv("big_vultest.csv")
gbig_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  1170 non-null   object
 1   input        1170 non-null   object
 2   output       1170 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 27.5+ KB


In [3]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './cdl_codebert_model_final'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
gbig_test_df = gbig_test_df.rename(columns={"input": "func", "output": "label"})
gbig_test_df = gbig_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
gbig_test_ds = Dataset.from_pandas(gbig_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

big_test_tok = gbig_test_ds.map(tokenize_fn, batched=True)
big_test_tok = big_test_tok.rename_column("label", "labels")
big_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=big_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on BigVul Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map:   0%|          | 0/1170 [00:00<?, ? examples/s]

C:\Users\user01\AppData\Local\Temp\ipykernel_26576\1867341218.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



=== Evaluation on BigVul Dataset ===
loss        : 2.8405
model_preparation_time: 0.0040
accuracy    : 0.5325
precision   : 0.5171
recall      : 0.9829
specificity : 0.0821
fpr         : 0.9179
f1          : 0.6777
mcc         : 0.1496
kappa       : 0.0650
mse         : 0.4675
mae         : 0.4675
auc         : 0.7430
runtime     : 11.9095
samples_per_second: 98.2410
steps_per_second: 12.3430


In [4]:
import pandas as pd
gdiv_test_df = pd.read_csv("diverse_test.csv")
gdiv_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532 entries, 0 to 1531
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   code_snip  1532 non-null   object
 1   output     1532 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 24.1+ KB


In [5]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './cdl_codebert_model_final'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
gdiv_test_df = gdiv_test_df.rename(columns={"code_snip": "func", "output": "label"})
gdiv_test_df= gdiv_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
gdiv_test_ds = Dataset.from_pandas(gdiv_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

div_test_tok = gdiv_test_ds.map(tokenize_fn, batched=True)
div_test_tok = div_test_tok.rename_column("label", "labels")
div_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=div_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on Diverse Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map:   0%|          | 0/1532 [00:00<?, ? examples/s]

C:\Users\user01\AppData\Local\Temp\ipykernel_26576\3336177793.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



=== Evaluation on Diverse Dataset ===
loss        : 2.7463
model_preparation_time: 0.0056
accuracy    : 0.5215
precision   : 0.5120
recall      : 0.9178
specificity : 0.1253
fpr         : 0.8747
f1          : 0.6573
mcc         : 0.0706
kappa       : 0.0431
mse         : 0.4785
mae         : 0.4785
auc         : 0.5649
runtime     : 15.7042
samples_per_second: 97.5530
steps_per_second: 12.2260


In [6]:
import pandas as pd
juliet_test_df = pd.read_csv("djuliet_test.csv")
juliet_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3152 entries, 0 to 3151
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   code_snip  3152 non-null   object
 1   output     3152 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 49.4+ KB


In [7]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './cdl_codebert_model_final'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
juliet_test_df = juliet_test_df.rename(columns={"code_snip": "func", "output": "label"})
juliet_test_df = juliet_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
juliet_test_ds = Dataset.from_pandas(juliet_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

juliet_test_tok = juliet_test_ds.map(tokenize_fn, batched=True)
juliet_test_tok = juliet_test_tok.rename_column("label", "labels")
juliet_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=juliet_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on Juliet Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map:   0%|          | 0/3152 [00:00<?, ? examples/s]

C:\Users\user01\AppData\Local\Temp\ipykernel_26576\3275667978.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



=== Evaluation on Juliet Dataset ===
loss        : 3.2348
model_preparation_time: 0.0109
accuracy    : 0.5079
precision   : 0.5040
recall      : 1.0000
specificity : 0.0159
fpr         : 0.9841
f1          : 0.6702
mcc         : 0.0894
kappa       : 0.0159
mse         : 0.4921
mae         : 0.4921
auc         : 0.6584
runtime     : 33.4897
samples_per_second: 94.1190
steps_per_second: 11.7650


In [8]:
import pandas as pd
reveal_test_df = pd.read_csv("Reveal_vultest.csv")
reveal_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2028 entries, 0 to 2027
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  2028 non-null   object
 1   input        2028 non-null   object
 2   output       2028 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 47.7+ KB


In [9]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './cdl_codebert_model_final'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame
reveal_test_df = pd.read_csv("Reveal_vultest.csv")

# Standardize column names and drop missing values
reveal_test_df = reveal_test_df.rename(columns={"input": "func", "output": "label"})
reveal_test_df = reveal_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
reveal_test_ds = Dataset.from_pandas(reveal_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

reveal_test_tok = reveal_test_ds.map(tokenize_fn, batched=True)
reveal_test_tok = reveal_test_tok.rename_column("label", "labels")
reveal_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=reveal_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on Reveal Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map:   0%|          | 0/2028 [00:00<?, ? examples/s]

C:\Users\user01\AppData\Local\Temp\ipykernel_26576\3829860020.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



=== Evaluation on Reveal Dataset ===
loss        : 1.0895
model_preparation_time: 0.0059
accuracy    : 0.5321
precision   : 0.5636
recall      : 0.2840
specificity : 0.7801
fpr         : 0.2199
f1          : 0.3777
mcc         : 0.0738
kappa       : 0.0641
mse         : 0.4679
mae         : 0.4679
auc         : 0.5354
runtime     : 24.8247
samples_per_second: 81.6930
steps_per_second: 10.2320


In [10]:
import pandas as pd
mix_test_df = pd.read_csv("mix_test_vultest.csv")
mix_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  2864 non-null   object
 1   input        2864 non-null   object
 2   output       2864 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 67.2+ KB


In [11]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './cdl_codebert_model_final'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======


# Standardize column names and drop missing values
mix_test_df = mix_test_df.rename(columns={"input": "func", "output": "label"})
mix_test_df = mix_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
mix_test_ds = Dataset.from_pandas(mix_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

mix_test_tok = mix_test_ds.map(tokenize_fn, batched=True)
mix_test_tok = mix_test_tok.rename_column("label", "labels")
mix_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=mix_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on mix Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map:   0%|          | 0/2864 [00:00<?, ? examples/s]

C:\Users\user01\AppData\Local\Temp\ipykernel_26576\3120241049.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



=== Evaluation on mix Dataset ===
loss        : 2.1113
model_preparation_time: 0.0060
accuracy    : 0.5426
precision   : 0.5317
recall      : 0.7137
specificity : 0.3715
fpr         : 0.6285
f1          : 0.6094
mcc         : 0.0907
kappa       : 0.0852
mse         : 0.4574
mae         : 0.4574
auc         : 0.5511
runtime     : 29.8970
samples_per_second: 95.7960
steps_per_second: 11.9740


In [12]:
import pandas as pd

fun_test_df = pd.read_json("test_512.json")
fun_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4216 entries, 0 to 4215
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  4216 non-null   object
 1   input        4216 non-null   object
 2   output       4216 non-null   int64 
 3   idx          4216 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 131.9+ KB


In [13]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './cdl_codebert_model_final'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
fun_test_df = fun_test_df.rename(columns={"input": "func", "output": "label"})
fun_test_df = fun_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
fun_test_ds = Dataset.from_pandas(fun_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

fun_test_tok = fun_test_ds.map(tokenize_fn, batched=True)
fun_test_tok = fun_test_tok.rename_column("label", "labels")
fun_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=fun_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on VULLM fol Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map:   0%|          | 0/4216 [00:00<?, ? examples/s]

C:\Users\user01\AppData\Local\Temp\ipykernel_26576\3114608965.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



=== Evaluation on VULLM fol Dataset ===
loss        : 2.7515
model_preparation_time: 0.0000
accuracy    : 0.5097
precision   : 0.5054
recall      : 0.9127
specificity : 0.1067
fpr         : 0.8933
f1          : 0.6505
mcc         : 0.0329
kappa       : 0.0194
mse         : 0.4903
mae         : 0.4903
auc         : 0.5536
runtime     : 45.1698
samples_per_second: 93.3370
steps_per_second: 11.6670
